In [189]:
import numpy as np 
import pandas as pd 
from scipy.stats import distributions as iid
import statsmodels.api as sm
from matplotlib import pyplot as plt 
from scipy.optimize import minimize_scalar

In [190]:
data = pd.read_stata('/Users/cassturk/Downloads/angrist-krueger91.dta')

In [191]:
data = data.head(10000)

In [192]:
#readying data
j =pd.get_dummies(data['yob'])

j = j.drop(columns = [1939])

m = data['yob'].astype(str)+data['qob'].astype(str)
l = pd.get_dummies(m)

n = pd.get_dummies(data['region'])

d = np.array([list(data['logwage'])])
a= np.array([list(data['edu'])])


#print(np.array(data['black']).T.shape)

b = np.r_[a,np.array(j).T, np.array([list(data['black'])]), np.array([list(data['smsa'])]), np.array([list(data['married'])]) ,np.array(n).T]

z = np.r_[np.array(l).T, np.array(j).T, np.array([list(data['black'])]), 
              np.array([list(data['smsa'])]), np.array([list(data['married'])]), np.array(n).T]

print(d.shape)
print(b.shape)
print(z.shape)

(1, 10000)
(21, 10000)
(60, 10000)


In [193]:
#getting values for covariates
j =pd.get_dummies(data['yob'])

m = data['yob'].astype(str)+data['qob'].astype(str)


l = pd.get_dummies(m)

l = l.drop(columns=['19393', '19391', '19392', '19394', '19303', '19313', '19323', 
                        '19333', '19343','19353', '19363','19373'])
j = j.drop(columns = [1939])

d = np.array([list(data['logwage'])])
a= np.array([list(data['edu'])])

b = np.r_[a,np.array(j).T, np.array([list(data['black'])]), np.array([list(data['smsa'])]), np.array([list(data['married'])]) ,np.array(n).T]

model = sm.OLS(d.T,b.T)
results = model.fit()
results.params

array([ 0.06185895,  0.04466469,  0.03536557,  0.04112089,  0.03544588,
        0.03493746,  0.0100583 , -0.01264055, -0.05430268, -0.05522192,
        4.04543096,  0.15040736,  0.20071592,  0.61664573,  0.69012796,
        0.33737827,  0.54841281,  0.56243703,  0.3914977 ,  0.5445062 ,
        0.35442527])

In [181]:
vals = [0.04466469,  0.03536557,  0.04112089,  0.03544588,
        0.03493746,  0.0100583 , -0.01264055, -0.05430268, -0.05522192,
        4.04543096,  0.15040736,  0.20071592,  0.61664573,  0.69012796,
        0.33737827,  0.54841281,  0.56243703,  0.3914977 ,  0.5445062 ,
        0.35442527]

def p_val(y, x, Z, beta0): 
    '''
    Compute p-value from an F test on \gamma = 0 
    
    Input: y, x, Z, beta0 (guess of true beta)
    
    Ouput: p-value 
    '''
    hmm = beta0
    beta1 = [hmm]
    for item in vals:
        beta1.append(item )
    beta1 = np.array([beta1])
    #print(np.array([y.squeeze()]))
    #print(y.shape)
    #print((beta1@x.T).T.shape)
    #print(Z.shape)
    
    
    yh = np.subtract(y , (beta1@x.T).T) # transform the LHS var
    #print(yh.shape)
    #print(yh)
    model = sm.OLS(yh, Z)
    lm = model.fit() # fit the linear model 
    #print(lm.params)
    return lm.f_pvalue # return p value from F test


In [182]:


def max_p(y, x, Z):
    '''
    Obtain the estimate beta0 that max p-value 
    
    Input: y, x, Z 
    
    Output: beta0, max p-value 
    '''
    beta_lst = list(np.arange(0, 2, 0.01)) # Try beta0 in this range 
    p_lst = [] 
    
    for i in range(len(beta_lst)): # get p-value for each corresponding beta0
        p_lst.append(p_val(y, x, Z, beta_lst[i])) 
    p_dict = dict(zip(beta_lst, p_lst))
    
    max_beta = max(p_dict, key=p_dict.get)
    max_pval = p_dict[max_beta] # maxinmal pval and beta 
    
    ci_beta = list(dict(filter(lambda item: item[1]>=0.05, p_dict.items())).keys()) 
    ci = [ci_beta[0], ci_beta[-1]] # confidence interval 
    
    return max_beta, max_pval, ci



In [183]:
#readying data
j =pd.get_dummies(data['yob'])

j = j.drop(columns = [1939])

m = data['yob'].astype(str)+data['qob'].astype(str)
l = pd.get_dummies(m)

n = pd.get_dummies(data['region'])

d = np.array([list(data['logwage'])])
a= np.array([list(data['edu'])])


#print(np.array(data['black']).T.shape)

b = np.r_[a,np.array(j).T, np.array([list(data['black'])]), np.array([list(data['smsa'])]), np.array([list(data['married'])]) ,np.array(n).T]

z = np.r_[np.array(l).T, np.array(j).T, np.array([list(data['black'])]), 
              np.array([list(data['smsa'])]), np.array([list(data['married'])]), np.array(n).T]

print(d.shape)
print(b.shape)
print(z.shape)


(1, 10000)
(21, 10000)
(60, 10000)


In [184]:

print(max_p(d.T, b.T, z.T))

b, pval, ci = max_p(d.T, b.T, z.T) # estimate beta
print(f'The estimated coefficient is {b}, the corresponding maximal p-value is {pval}')


(0.06, 0.9583037698654765, [0.03, 0.09])
The estimated coefficient is 0.06, the corresponding maximal p-value is 0.9583037698654765


In [185]:
def two_sls(y, X, Z):
    '''
    Estimate 2 stage least squares given data on y, X and Z.
    
    Inputs:
    -------
    y (Nx1 np.array)
    X (Nx1 np.array)
    Z (Nxl np.array)
    
    Returns
    -------
    beta_hat, vb_beta_hat
    '''
    y = np.array(y)
    X = np.array(X)
    Z = np.array(Z)
    ZX = np.transpose(Z)@X
    ZX_inv = np.linalg.pinv(ZX)#using pseuodo-inverse
    Zy = np.transpose(Z)@y
    ZZ = np.transpose(Z)@Z
    ZZ_inv = np.linalg.pinv(ZZ)
    #Beta_IV2SLS = ZX_inv@ZY
    beta_hat = np.linalg.pinv(np.transpose(ZX)@ZZ_inv@ZX)@(np.transpose(ZX)@ZZ_inv@Zy)
    e = y - X@beta_hat
    SSR = np.transpose(e)@e
    N = X.shape[0]
    vcv = (SSR/N)*np.linalg.pinv(np.transpose(ZX)@ZZ_inv@ZX)
    K = X.shape[1]
    se_beta_hat = np.zeros(K)
    for i in range(K):
          se_beta_hat[i] = np.sqrt(vcv[i,i])
            
    return beta_hat, se_beta_hat



In [195]:
b_2sls, se_2sls = two_sls(d.T, b.T, z.T)
print(f"The estimated coefficient from 2SLS is {b_2sls[0][0]}, and the confidence interval is {b_2sls[0][0] - 1.96*se_2sls[0], b_2sls[0][0] + 1.96*se_2sls[0]}.")

print(d.shape)
print(b.shape)
print(z.shape)

The estimated coefficient from 2SLS is 0.09708617398871411, and the confidence interval is (0.017785438763933598, 0.17638690921349462).
(1, 10000)
(21, 10000)
(60, 10000)
